In [17]:
import pandas as pd
import numpy as np
from itertools import chain
from collections import defaultdict, Counter

# Answer to Question 2 & 3: 
## Create a DataFrame with the missing names filled in.
## Find the top 10 major project theme
---

### Step_1: Make a new DataFrame with columns "code" and "name", which contain the complete mapped project code and name.

 * ### __Inspect the data__

In [18]:
df = pd.read_json('data/world_bank_projects.json')

In [19]:
df = df.loc[:,['mjtheme','mjtheme_namecode','mjthemecode']]

In [20]:
df.head()

,mjtheme,mjtheme_namecode,mjthemecode
0,[Human development],"[{'code': '8', 'name': 'Human development'}, {...","8,11"
1,"[Economic management, Social protection and ri...","[{'code': '1', 'name': 'Economic management'},...","1,6"
2,"[Trade and integration, Public sector governan...","[{'code': '5', 'name': 'Trade and integration'...","5,2,11,6"
3,"[Social dev/gender/inclusion, Social dev/gende...","[{'code': '7', 'name': 'Social dev/gender/incl...","7,7"
4,"[Trade and integration, Financial and private ...","[{'code': '5', 'name': 'Trade and integration'...","5,4"


In [21]:
data = df.mjtheme_namecode.values
data[:3]

array([[{'code': '8', 'name': 'Human development'}, {'code': '11', 'name': ''}],
       [{'code': '1', 'name': 'Economic management'}, {'code': '6', 'name': 'Social protection and risk management'}],
       [{'code': '5', 'name': 'Trade and integration'}, {'code': '2', 'name': 'Public sector governance'}, {'code': '11', 'name': 'Environment and natural resources management'}, {'code': '6', 'name': 'Social protection and risk management'}]], dtype=object)

* ### __Merge all the dictionaries in the array into one dictionary: "dict3", since they have identical keys.__

In [22]:
# merge the dictionaries above, 
# add all code into a list as the values of key 'code' in order.
# add all name into a list as the values of key 'name' in order.
# code and name are added in order, so they are correctly mapped. 
# dict3 has two keys 'code' and 'name'.
dict3 = defaultdict(list)
for n in range(data.shape[0]):
    for i in range(len(data[n])):
        for k, v in data[n][i].items():
            dict3[k].append(v)

* ### __ Replace the missing name with NaN__

In [24]:
# replace the missing name '' as a NaN value.
for n,i in enumerate(dict3['name']):
    if i == '':
        dict3['name'][n]=np.nan
dict3['name'][:8]

['Human development',
 nan,
 'Economic management',
 'Social protection and risk management',
 'Trade and integration',
 'Public sector governance',
 'Environment and natural resources management',
 'Social protection and risk management']

* ### __Check the frequency of name and code__

In [25]:
Counter(dict3['name'])

Counter({'Human development': 197,
         nan: 122,
         'Economic management': 33,
         'Social protection and risk management': 158,
         'Trade and integration': 72,
         'Public sector governance': 184,
         'Environment and natural resources management': 223,
         'Social dev/gender/inclusion': 119,
         'Financial and private sector development': 130,
         'Rural development': 202,
         'Urban development': 47,
         'Rule of law': 12})

In [26]:
Counter(dict3['code'])

Counter({'1': 38,
         '10': 216,
         '11': 250,
         '2': 199,
         '3': 15,
         '4': 146,
         '5': 77,
         '6': 168,
         '7': 130,
         '8': 210,
         '9': 50})

__Note:__ The frequency of each code and frequency of each project name doesn't match.
They will match after the missing project name is filled.

* ### __ Make the Reference DataFrame. It contain the complete mapped name and code.__ 

In [27]:
projects_df = pd.DataFrame(dict3)

In [28]:
projects_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1499 entries, 0 to 1498
Data columns (total 2 columns):
code    1499 non-null object
name    1377 non-null object
dtypes: object(2)
memory usage: 23.5+ KB


In [29]:
projects_df = projects_df.drop_duplicates()
projects_df = projects_df.dropna()
projects_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 0 to 252
Data columns (total 2 columns):
code    11 non-null object
name    11 non-null object
dtypes: object(2)
memory usage: 264.0+ bytes


In [30]:
projects_df

,code,name
0,8,Human development
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration
5,2,Public sector governance
6,11,Environment and natural resources management
8,7,Social dev/gender/inclusion
11,4,Financial and private sector development
18,10,Rural development
53,9,Urban development


In [32]:
projects_df.set_index('code')

,name
code,
8,Human development
1,Economic management
6,Social protection and risk management
5,Trade and integration
2,Public sector governance
11,Environment and natural resources management
7,Social dev/gender/inclusion
4,Financial and private sector development
10,Rural development


In [33]:
projects_df = projects_df.sort_index()

In [35]:
projects_df.to_csv('complete_projects_codename.csv')

### Step_2: Filling the missing name

* ### __ Import the reference DataFrame.__ 

In [36]:
# The reference DataFrame.
projects_df = pd.read_csv('complete_projects_codename.csv', index_col=1)
projects_df = projects_df.loc[:,['name']]
projects_df

,name
code,
8,Human development
1,Economic management
6,Social protection and risk management
5,Trade and integration
2,Public sector governance
11,Environment and natural resources management
7,Social dev/gender/inclusion
4,Financial and private sector development
10,Rural development


* ### __ Fill the missing names in the "mjtheme_namecode" column .__ 

In [37]:
data = df.mjtheme_namecode.values

In [38]:
# Check every row's mjtheme_namecode attribute, fill the missing project 'name'
# Look up the reference DataFrame: projects_df, 
# find the correspoinding project 'name' to the project 'code' from the projects_df DataFrame.
for n in range(data.shape[0]):
    for i in range(len(data[n])):
        if data[n][i]['name'] == ''
            data[n][i]['name'] = projects_df.loc[int(data[n][i]['code'])].values[0]

* ### __ Check the frequency of name and code in "mjtheme_namecode" column__ 

In [39]:
# Merge the dictionaries above, 
# add all code into a list as the values of key 'code' in order.
# Add all name into a list as the values of key 'name' in order.
# Code and name are added in order, so they are correctly mapped. 
# Dict3 has two keys 'code' and 'name'.
dict3 = defaultdict(list)
for n in range(data.shape[0]):
    for i in range(len(data[n])):
        for k, v in data[n][i].items():
            dict3[k].append(v)

In [40]:
# Frequency of each proejct name.
Counter(dict3['name'])

Counter({'Economic management': 38,
         'Environment and natural resources management': 250,
         'Financial and private sector development': 146,
         'Human development': 210,
         'Public sector governance': 199,
         'Rule of law': 15,
         'Rural development': 216,
         'Social dev/gender/inclusion': 130,
         'Social protection and risk management': 168,
         'Trade and integration': 77,
         'Urban development': 50})

In [41]:
# Frequency of each project code.
Counter(dict3['code'])

Counter({'1': 38,
         '10': 216,
         '11': 250,
         '2': 199,
         '3': 15,
         '4': 146,
         '5': 77,
         '6': 168,
         '7': 130,
         '8': 210,
         '9': 50})

**The frequency of each project code and frequency of each project name does match.**

* ### __ Update the reference DataFrame with frequency __ 

In [42]:
# Final complete mjtheme projects name-code-frequency DataFrame
projects_df_sorted = projects_df.sort_index()
projects_df_sorted['frequency'] = pd.Series([38,199,15,146,77,168,130,210,50,216,250],index = projects_df_sorted.index)
projects_df_sorted

,name,frequency
code,,
1,Economic management,38
2,Public sector governance,199
3,Rule of law,15
4,Financial and private sector development,146
5,Trade and integration,77
6,Social protection and risk management,168
7,Social dev/gender/inclusion,130
8,Human development,210
9,Urban development,50


### Answer to Q2: top 10 major proejct theme.

In [46]:
Counter(dict3['name']).most_common(10)

[('Environment and natural resources management', 250),
 ('Rural development', 216),
 ('Human development', 210),
 ('Public sector governance', 199),
 ('Social protection and risk management', 168),
 ('Financial and private sector development', 146),
 ('Social dev/gender/inclusion', 130),
 ('Trade and integration', 77),
 ('Urban development', 50),
 ('Economic management', 38)]

### Answer to Q3: Create a DataFrame with the missing names filled in.

In [44]:
# DataFrame with the missing names filled in.
df['mjtheme_namecode'] = pd.Series(data)
df.mjtheme_namecode.head(10)

0    [{'code': '8', 'name': 'Human development'}, {...
1    [{'code': '1', 'name': 'Economic management'},...
2    [{'code': '5', 'name': 'Trade and integration'...
3    [{'code': '7', 'name': 'Social dev/gender/incl...
4    [{'code': '5', 'name': 'Trade and integration'...
5    [{'code': '6', 'name': 'Social protection and ...
6    [{'code': '2', 'name': 'Public sector governan...
7    [{'code': '11', 'name': 'Environment and natur...
8    [{'code': '10', 'name': 'Rural development'}, ...
9    [{'code': '2', 'name': 'Public sector governan...
Name: mjtheme_namecode, dtype: object

__Notice__: there's missing project name in the mjtheme column. The missing names could be filled in by the same method. 